<center><h1>how to make the code fast</h1></center>

<br/>

### The golden rule of code optimization

<br/><br/><br/><br/><br/><br/><br/><br/><br/><br/><br/><br/><br/><br/><br/><br/><br/><br/>






# DON'T DO IT


<br/><br/>
<br/><br/>
<br/><br/>
<br/><br/>
<br/><br/>



## yet





### make sure your code does _exactly_ what you want and does so _correctly_ before even thinking about making it fast.

### Then:
- vectorize
- reduce function calls + other cleverness
- present an offering to our savior, Numba, and hope they smile favorably upon your code

**working example**: HW 1 problems 3 and 3.5 - adapted from Maryn Carlson's code

In [ ]:
import numpy as np
def binary_digits(n):
    if n == 1:
        return [[0], [1]]
    return [[*row, i] for row in binary_digits(n-1) for i in range(2)]

In [ ]:
bin_mat_small = binary_digits(3)
bin_mat_small

In [ ]:
bin_mat_10 = binary_digits(10)

Problem: count the number of values that are 1 in rows i and j of a binary matrix.

In [ ]:
def create_counts_matrix_purepython(binMat):
    nstates = len(binMat)
    n = len(binMat[0])
    counts = []

    for i in range (nstates):
        counts.append([])
        for j in range(nstates):
            cell_val = 0
            for row_idx in range(n):
                if binMat[i][row_idx] == 1 and binMat[j][row_idx] == 1:
                    cell_val += 1
            counts[i].append(cell_val)
    return counts

In [ ]:
create_counts_matrix_purepython(bin_mat_small)

In [ ]:
test_result = create_counts_matrix_purepython(bin_mat_10)

### profiling - "this took a while, but how long, exactly?"

In [ ]:
%%timeit -r 3 -n 3
create_counts_matrix_purepython(bin_mat_10)

##### tqdm aside:

In [ ]:
from tqdm.notebook import tqdm
def create_counts_matrix_tqdm(binMat):
    nstates = len(binMat)
    n = len(binMat[0])
    counts = []

    for i in tqdm(range(nstates)):
        counts.append([])
        for j in range(nstates):
            cell_val = 0
            for row_idx in range(n):
                if binMat[i][row_idx] == 1 and binMat[j][row_idx] == 1:
                    cell_val += 1
            counts[i].append(cell_val)
    return counts

In [ ]:
_ = create_counts_matrix_tqdm(bin_mat_10)

In [ ]:
%load_ext line_profiler

In [ ]:
%lprun -f create_counts_matrix_purepython create_counts_matrix_purepython(bin_mat_10)

alternatives: cProfile + snakeviz (*show plot*), scalene (I can't read it but it's apparently good)
https://coderzcolumn.com/tutorials/python/snakeviz-visualize-profiling-results-in-python is a good intro to snakeviz

### level 1: vectorization

Vectorization just means converting for loops to numpy operations. As a quick example, the formula for the allele frequency in the n+1st generation under selection is $$p' = p + sp(1-p)/2$$. Two ways to compute a vector of allele frequencies in generation n+1 given a vector of allele frequencies in generation n and a selection coefficient are:

In [ ]:
import numpy as np
p = np.random(size=100)
s = 0.01
p_prime = np.zeros_like(p)
for i in range(p.shape[0]):
    p_prime[i] = p[i] + s*p[i]*(1-p[i])
    
    
p_prime = p + s*p*(1-p)

The second is faster, easier to understand, and closer to the formula above in appearance.

Now, let's apply this to create_counts_matrix:

In [ ]:
def create_counts_matrix_purepython(binMat):
    nstates = len(binMat)
    n = len(binMat[0])
    counts = []

    for i in range (nstates):
        counts.append([])
        for j in range(nstates):
            cell_val = 0
            for row_idx in range(n):
                if binMat[i][row_idx] == 1 and binMat[j][row_idx] == 1:
                    cell_val += 1
            counts[i].append(cell_val)
    return counts

In [ ]:
def create_counts_matrix_level1(binMat):
    nstates, n = binMat.shape
    counts = np.zeros((nstates, nstates), dtype=np.int8)

    for i in range(nstates):
        for j in range(nstates):
            prod = binMat[i,:]*binMat[j,:]
            nThrees  = np.sum (prod == 1)
            counts[i, j] = nThrees
    return counts

In [ ]:
bin_arr_10 = np.array(bin_mat_10)

In [ ]:
%%timeit -r 2 -n 1
create_counts_matrix_level1(bin_arr_10)

In [ ]:
%lprun -f create_counts_matrix_level1 create_counts_matrix_level1(bin_arr_10)

### level 2: reduce calls + cleverness

how do we deal with that gosh darn for loop??

In [ ]:
def create_counts_matrix_level2(binMat):
    nstates, n = binMat.shape
    counts = np.zeros((nstates, nstates), dtype=np.int8)

    for i in range (nstates):
        temp_prod = np.zeros_like(binMat)
        for j in range(nstates):
            temp_prod[j, :] = binMat[i,:]*binMat[j,:]
        nThrees  = np.sum(temp_prod == 1, axis=1)
        counts[i, :] = nThrees
    return counts

In [ ]:
%%timeit -r 3 -n 2
create_counts_matrix_level2(bin_arr_10)

In [ ]:
%lprun -f create_counts_matrix_level2 create_counts_matrix_level2(bin_arr_10)

In [ ]:
def create_counts_matrix_level2_2(binMat):
    nstates, n = binMat.shape
    counts = np.zeros((nstates, nstates), dtype=np.int8)

    for i in range (nstates) :
        plus = binMat[i,:]*binMat #broadcasting!!
        nThrees  = np.sum(plus == 1, axis=1)
        counts[i, :] = nThrees
    return counts

In [ ]:
%%timeit -n 10
create_counts_matrix_level2_2(bin_arr_10)

In [ ]:
%lprun -f create_counts_matrix_level2_2 create_counts_matrix_level2_2(bin_arr_10)

In [ ]:
def create_counts_matrix_level2_3(binMat):
    return np.einsum("ij, kj -> ik", binMat, binMat)

In [ ]:
np.all(create_counts_matrix_level2_3(bin_mat_10) == create_counts_matrix_level2_2(bin_arr_10))

In [ ]:
%%timeit -n 50
create_counts_matrix_level2_3(bin_arr_10)

### level 3: praise Numba

numba is straight up black magic

like all good black magic, it's a bit finnicky:
- don't use numba for certain stuff (non-numerical data, randomness)
- doesn't produce even remotely useful error messages.

In [ ]:
from numba import njit

In [ ]:
def create_counts_matrix_purepython(binMat):
    nstates, n = binMat.shape
    counts = []
    for i in range (nstates):
        counts.append([])
        for j in range(nstates):
            cell_val = 0
            for row_idx in range(n):
                if binMat[i, row_idx] == 1 and binMat[j, row_idx] == 1:
                    cell_val += 1
            counts[i].append(cell_val)
    return counts

In [ ]:
@njit(cache=True)
def create_counts_matrix_numba(binMat):
    nstates, n = binMat.shape
    counts = np.zeros((nstates, nstates), dtype=np.int8)

    for i in range (nstates) :
        for j in range(nstates):
            cell_val = 0
            for row_idx in range(n):
                if binMat[i, row_idx] == 1 and binMat[j, row_idx] == 1:
                    cell_val += 1
            counts[i, j] = cell_val
    return counts

In [ ]:
_ = create_counts_matrix_numba(bin_arr_10)

In [ ]:
%%timeit -n 50
create_counts_matrix_numba(bin_arr_10)

In [ ]:
%lprun -f create_counts_matrix_numba create_counts_matrix_numba(bin_arr_10)

In [ ]:
@njit(cache=True)
def create_counts_matrix_numba_2(binMat):
    nstates, n = binMat.shape
    counts = np.zeros((nstates, nstates), dtype=np.int8)
    for i in range (nstates) :
        for j in range(i, nstates):
            cell_val = 0
            for row_idx in range(n):
                if binMat[i, row_idx] == 1 and binMat[j, row_idx] == 1:
                    cell_val += 1
            counts[j, i] = counts[i, j] = cell_val
    return counts

In [ ]:
_ = create_counts_matrix_numba_2(bin_arr_10)

In [ ]:
%%timeit -n 50
create_counts_matrix_numba_2(bin_arr_10)

In [ ]:
@njit(cache=True)
def create_counts_matrix_numba_3(binMat):
    nstates, n = binMat.shape
    counts = np.zeros((nstates, nstates), dtype=np.int8)
    for i in range (nstates) :
        for j in range(i, nstates):
            cell_val = 0
            for row_idx in range(n):
                if binMat[i, row_idx]*binMat[j, row_idx] == 1:
                    cell_val += 1
            counts[j, i] = counts[i, j] = cell_val
    return counts

In [ ]:
_ = create_counts_matrix_numba_3(bin_arr_10)

In [ ]:
%%timeit -n 50
create_counts_matrix_numba_3(bin_arr_10)

#### what have we learned

- <h4>write code that works first!</h4>
- if it's slow, use a profiler (e.g. line_profiler, cProfile + snakeviz) to figure out how slow it is and where it's slow
- vectorize, rewrite to minimize function calls, @njit

### Reproducibility and other good research practices

#### how to use git
<br/><br/><br/><br/><br/><br/>
<br/><br/><br/><br/><br/><br/>
<br/><br/><br/><br/><br/><br/>
ok I lied I'm not going to teach you anything about git. why? Git has actively impeded my research progress on several occasions. Unless you're in Meren's lab, which is impossible b/c he's leaving I think, projects you are involved in are going to be coded by at most two people. You do not need git if you are the only person coding anything.
The one aspect of git I have used as a solo coder is if you want to update stuff on a remote directory/cluster/etc. - it can be nice to make all your local changes than just git pull on the cluster.
Ok but what about versioning/reverting/etc.????? Box! Putting all your stuff on box is good in case your computer breaks. Also, thanks to the University of Chicago, we have this wonderful thing called "version history" that will *automatically* keep track of basically all the changes you make to something. No need to git pull git update git push whatever.

### argparse

sometimes, you cannot run your code inside PyCharm, but must run it from the command line. The two primary instances of this are:
- when publishing, people often want a command line tool. idrk why but they do.
- for cluster work it's somewhere between much easier and the only way to get jobs to run.

you can use sys.argv and make your command something like python science.py 4 10 "linear" 500 1e4 "fast" 8

or use argparse! Python's built in library for, unsurprisingly, parsing command line arguments.

How argparse works: 3 easy steps.
    
1. set up ArgumentParser

In [ ]:
import argparse
parser = argparse.ArgumentParser()

2. add arguments

In [ ]:
parser.add_argument("filename", type=argparse.FileType("rb"), help="input file")
parser.add_argument("-n", "--number", type=int, default=5, help="an optional integer")
parser.add_argument("--print_this_stuff", nargs="*", help="prints all the extra args you put in")

3. parse args

In [ ]:
args = parser.parse_args()

#args now has a variable for each argument:
print(args.filename)
print(args.number)
for val in args.print_this_stuff:
    print(val)

#also the documentation is built-in! -h

Because it is a well-written module, argparse can handle whatever stuff you might want out of your inputs. Different types, required/optional arguments, mutually exclusive groups (e.g. "verbose" mode vs. "quiet" mode) - use group = parser.add_mutually_exclusive_group() and then group.add_argument()

#### snakemake

automated workflow creation! no more running 10 python scripts in a row oh wait I forgot I updated the 6th one now I have to rerun everything again oh wait I ran these two out of order so it didn't run on the latest dataset time to rerun everything etc

In [ ]:
SAMPLES = ["A", "B"]


rule all:
    input:
        "plots/quals.svg"


rule bwa_map:
    input:
        "data/genome.fa",
        "data/samples/{sample}.fastq"
    output:
        "mapped_reads/{sample}.bam"
    shell:
        "bwa mem {input} | samtools view -Sb - > {output}"


rule samtools_sort:
    input:
        "mapped_reads/{sample}.bam"
    output:
        "sorted_reads/{sample}.bam"
    shell:
        "samtools sort -T sorted_reads/{wildcards.sample} "
        "-O bam {input} > {output}"


rule samtools_index:
    input:
        "sorted_reads/{sample}.bam"
    output:
        "sorted_reads/{sample}.bam.bai"
    shell:
        "samtools index {input}"


rule bcftools_call:
    input:
        fa="data/genome.fa",
        bam=expand("sorted_reads/{sample}.bam", sample=SAMPLES),
        bai=expand("sorted_reads/{sample}.bam.bai", sample=SAMPLES)
    output:
        "calls/all.vcf"
    shell:
        "bcftools mpileup -f {input.fa} {input.bam} | "
        "bcftools call -mv - > {output}"


rule plot_quals:
    input:
        "calls/all.vcf"
    output:
        "plots/quals.svg"
    script:
        "scripts/plot-quals.py"

![snakemake_dag](https://snakemake.readthedocs.io/en/stable/_images/dag_call.png)

support for clusters (with special SLURM support!) is built-in as well